In [1]:
import dlib
import cv2
from imutils import face_utils
from threading import Thread
#import threading
import os
import time 
import glob
from os.path import basename, splitext


In [2]:

class CSave:
    def __init__(self,path):
        self.counter = 1
        self.path = path
        self.DataFile=""
        pass
    def InitFil(self,FileName):
        os.mkdir(self.path+FileName)
    
    def SaveTxt (self,FileName,Id):
        f = open(FileName, 'w')
        f.write(str(Id))
        f.close()
    def ReadTxt (self,FileName):
        f = open(FileName, 'r')
        a = int(f.read())
        f.close()
        return a
    
    
    def SaveImageSq(self,frame,v,Lab):
        LabIm = Lab+"_"+str(self.counter)
        cv2.imwrite(self.path+LabIm+".jpg",frame[v[2]:v[2]+v[4],v[1]:v[1] + v[3]])
        print (LabIm)
        self.counter +=1
        
    def SaveImageCam(self,frame,Lab):
        LabIm = Lab+"_"+str(self.counter)
        cv2.imwrite(self.path+LabIm+".jpg",frame)
        print (LabIm)
        self.counter +=1
        
    def CreateFileCsv (self,Label='Data',entetes = ['Id','Nom','Prenom','Age','Niveau','Classe']):
        self.DataFile = self.path+Label+'.csv'
        f = open(self.DataFile, 'a')
        ligneEntete = ";".join(entetes) + "\n"
        f.write(ligneEntete)
        f.close()
        print("Le fichier de donnee a ete cree")
        
    def AddDataCsv(self,data):
       
        if (len(self.DataFile)==0):
            valeurs = [data['Id'],data['Nom'], data['Prenom'], data['Age'], data['Niveau'], data['Classe']]
            filepath = glob.glob(self.path+'*.csv')[0]
            DataFile = self.path+splitext(basename(filepath))[0]+'.csv'
            print(DataFile)
            f = open(DataFile, 'a')
            ligne = ";".join(valeurs) + "\n"
            f.write(ligne)
            f.close()
            self.Label = data['Id']
            print("les donnees sont ajoutees")
        else:
            valeurs = [data['Id'],data['Nom'], data['Prenom'], data['Age'], data['Niveau'], data['Classe']]
            f = open(self.DataFile, 'a')
            ligne = ";".join(valeurs) + "\n"
            f.write(ligne)
            f.close()
            self.Label = data['Id']
            print("les donnees sont ajoutees")

class Camera:
    
    def __init__(self):
        print("creation de la camera")
        self.cap = cv2.VideoCapture(0)
  
    def CaptureFrame(self):
        self.ret, self.frame = self.cap.read()
        if self.ret==False:
            print("verifier votre camera")
        else: 
            return self.frame
        
    def Display(self,title='Video Capture'):
        cv2.imshow(title,self.frame)
        
    def __del__(self):
        self.cap.release()
        cv2.destroyAllWindows()
        

class Detection: 
    
    def __init__(self):
        print("creation de l'objet detection")
        self.detector = dlib.get_frontal_face_detector()
        pass
    
    def LoadPredictor(self,data): 
        self.predictor = dlib.shape_predictor(data)
        
    def DetectFaces(self,frame):
        self.vect=[]
        self.gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        self.rects = self.detector(self.gray, 1)
        for (i, rect) in enumerate(self.rects):
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            if (i==0):
                self.vect = [x,y,w,h]
                
    def DetectFace(self,frame):
        self.vect=[]
        self.gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        self.rects = self.detector(self.gray, 1)
        if (len(self.rects) !=0):
            (x, y, w, h) = face_utils.rect_to_bb(self.rects[0])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            self.vect = [1,x,y,w,h]
        else:
            cv2.putText(frame, 'Visage non reconnu',(200,200) ,cv2.FONT_HERSHEY_TRIPLEX, 0.8, (0, 255, 0))
            self.vect = [0]
            
    def Detectlandmarks(self,frame,dots_size=5):
        self.vect=[]
        self.gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        self.rects = self.detector(self.gray, 1)
        for (i, rect) in enumerate(self.rects):
            shape = self.predictor(self.gray, rect)
            shape = face_utils.shape_to_np(shape)
            for (x, y) in shape:
                cv2.circle(frame, (x, y), dots_size, (0, 255, 0), -1)
            if (i==0):
                self.vect = [x,y,w,h]
                
    def DetectlandmarksRectrectangle(self,frame,dots_size=5):
        self.vect=[]
        self.gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        self.rects = self.detector(self.gray, 1)
        for (i, rect) in enumerate(self.rects):
            shape = self.predictor(self.gray, rect)
            shape = face_utils.shape_to_np(shape)
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            for (x, y) in shape:
                cv2.circle(frame, (x, y), dots_size, (0, 255, 0), -1)
            if (i==0):
                self.vect = [x,y,w,h]
     

# Programme principal

In [3]:
FileCsv = CSave("Data/")
id = FileCsv.ReadTxt("Id.txt")
if (id == 1):
    FileCsv.CreateFileCsv("DataEcole")
    
cam = Camera()
det = Detection()
det.LoadPredictor('shape_predictor_68_face_landmarks.dat')

while(True):
    print("Voulez-vous ajouter un nouvel étudiant ? oui/non :")
    rep =input()

    if (rep.lower()=="oui"):
        
        PersonalData = {}
        print("Entrez les données de l'etudiant :")
        PersonalData["Id"] = "Id_"+str(id)
        PersonalData["Nom"] = input("Nom :") 
        PersonalData["Prenom"] = input("Prenom :")
        PersonalData["Age"] = input("Age :")
        PersonalData["Niveau"] = input("Niveau d'etude :")
        PersonalData["Classe"] = input("Classe :")
        
        FileCsv.AddDataCsv(PersonalData)

        FileCsv.InitFil(PersonalData["Id"])
        
        SaveIm = CSave("Data/"+PersonalData["Id"]+"/")
       
        while(True):
            frame = cam.CaptureFrame()
            det.DetectFace(frame)
            v=det.vect
            #det.Detectlandmarks(frame)
            #det.DetectlandmarksRectrectangle(frame,1)
            cam.Display()
            key = cv2.waitKey(10)
            if key & 0xFF == ord('q'):
                break
            if (key & 0xFF == ord('w') and v[0]==1):
                Lab = FileCsv.Label
                SaveIm.SaveImageSq(frame,v,Lab)
        id+=1
    elif(rep.lower()=="non"):
        print("Fin.")
        break
    else:
        print("Exemple d'usage : oui/non..")

del cam

FileCsv.SaveTxt("Id.txt",id)




Le fichier de donnee a ete cree
creation de la camera
creation de l'objet detection
Voulez-vous ajouter un nouvel étudiant ? oui/non :
non
Fin.
